# Assertion RLA

## Overview of the assertion audit tool

The tool requires as input:

+ audit-specific and contest-specific parameters, such as
    - whether to sample with or without replacement
    - the name of the risk function to use, and any parameters it requires
    - a risk limit for each contest to be audited
    - the social choice function for each contest, including the number of winners
    - candidate identifiers
+ a ballot manifest
+ a random seed
+ a file of cast vote records
+ reported results for each contest
+ json files of assertions for IRV contests (one file per IRV contest)
+ human reading of voter intent from the paper ballots selected for audit

The tool helps select ballots for audit, and reports when the audit has found sufficiently strong evidence to stop.

The tool exports a log of all the audit inputs except the CVR file, but including the auditors' manually determined voter intent from the audited ballots.

In [1]:
from __future__ import division, print_function

import math
import json
import warnings
import numpy as np
import pandas as pd

from collections import OrderedDict
from IPython.display import display, HTML

from cryptorandom.cryptorandom import SHA256
from cryptorandom.sample import sample_by_index

from assertion_audit_utils import \
    Assertion, Assorter, CVR, TestNonnegMean,\
    check_audit_parameters, write_audit_parameters
from dominion_tools import \
    prep_dominion_manifest, write_ballots_sampled,\
    read_dominion_cvrs


# Audit parameters.

* `seed`: the numeric seed for the pseudo-random number generator used to draw sample 
* `replacement`: whether to sample with replacement. If the sample is drawn with replacement, gamma must also be specified.
* `risk_function`: the function to be used to measure risk. Options are `kaplan_markov`,`kaplan_wald`,`kaplan_kolmogorov`,`wald_sprt`,`kaplan_martingale`. Not all risk functions work with every social choice function. `wald_sprt` applies only to plurality contests.
* `g`: a parameter to hedge against the possibility of observing a maximum overstatement. Require $g \in [0, 1)$ for `kaplan_markov` and `kaplan_wald`
* `N_ballots`: an upper bound on the number of ballots cast in the contest. This should be derived independently of the voting system.

----

* `cvr_file`: filename for CVRs (input)
* `manifest_file`: filename for ballot manifest (input)
* `assertion_file`: filename of assertions for IRV contests, in RAIRE format
* `mvr_file`: filename for manually ascertained votes from sampled ballots (input)
* `log_file`: filename for audit log (output)

----

* `error_rates`: dict of expected error rates. The keys are
    + `o1_rate`: expected rate of 1-vote overstatements. Recommended value $\ge$ 0.001 if there are hand-marked ballots. Larger values increase the initial sample size, but make it more likely that the audit will conclude in a single round if the audit finds errors
    + `o2_rate`: expected rate of 2-vote overstatements. Recommended value 0.
    + `u1_rate`: expected rate of 1-vote understatements. Recommended value 0.
    + `u2_rate`: expected rate of 2-vote understatements. Recommended value 0.

* `contests`: a dict of contest-specific data 
    + the keys are unique contest identifiers for contests under audit
    + the values are dicts with keys:
        - `risk_limit`: the risk limit for the audit of this contest
        - `ballots_cast`: an upper bound on the number of cast ballots that contain the contest
        - `choice_function`: `plurality`, `supermajority`, or `IRV`
        - `n_winners`: number of winners for majority contests. (Multi-winner IRV not supported; multi-winner super-majority is nonsense)
        - `share_to_win`: for super-majority contests, the fraction of valid votes required to win, e.g., 2/3.
        - `candidates`: list of names or identifiers of candidates
        - `reported_winners` : list of identifier(s) of candidate(s) reported to have won. Length should equal `n_winners`.
        - `assertion_file`: filename for a set of json descriptors of Assertions (see technical documentation) that collectively imply the reported outcome of the contest is correct. Required for IRV; ignored for other social choice functions

In [2]:
seed = 12345678901234567890  # use, e.g., 20 rolls of a 10-sided die. Seed doesn't have to be numeric
replacement = True  # Sampling without replacement isn't implemented
risk_function = "kaplan_martingale"
g=0.1
N_ballots = 300000

In [3]:
cvr_file = './Data/CvrExport.json'
manifest_file = './Data/ballot manifest-Sample.xlsx'
mvr_file = './Data/mvr.csv'
log_file = './Data/log.json'

In [4]:
error_rates = {'o1_rate':0.002,      # expect 2 1-vote overstatements per 1000 ballots in the CVR stratum
               'o2_rate':0,          # expect 0 2-vote overstatements
               'u1_rate':0,          # expect 0 1-vote understatements
               'u2_rate':0}          # expect 0 2-vote understatements

In [5]:
# contests to audit
contests = {'mayor':{'risk_limit':0.05,
                     'choice_function':'IRV',
                     'n_winners':1,
                     'candidates':['Alice','Bob','Cindy'],
                     'reported_winners' : ['Alice'],
                     'assertion_file' : './Data/SF2019Nov8Assertions.json'
                    }
           }
"""            'city_council':{'risk_limit':0.05,
                     'choice_function':'plurality',
                     'n_winners':3,
                     'candidates':['Doug','Emily','Frank','Gail','Harry'],
                     'reported_winners' : ['Doug', 'Emily', 'Frank']
                    },
            'measure_1':{'risk_limit':0.05,
                     'choice_function':'supermajority',
                     'share_to_win':2/3,
                     'n_winners':1,
                     'candidates':['yes','no'],
                     'reported_winners' : ['yes']
                    }                  
           }
"""

In [6]:
check_audit_parameters(risk_function, g, error_rates, contests)
write_audit_parameters(log_file, seed, replacement, risk_function, g, N_ballots, error_rates, contests)

## Find audit parameters and conduct audit

* For each contest:
    - find claimed outcome by applying SCF to CVRs
    - complain if claimed outcome disagrees with reported outcome
    - construct assertions that imply contest outcome is correct
    - for each assertion:
        + find generalized diluted margin
        
* Find initial (incremental) sample size from smallest diluted margin, for the sampling plan
    - Complain if expected error rates imply any assertion is incorrect

* For each assertion:
    - Initialize discrepancy counts to zero (o1, o2, u1, u2)
    - Initialize measured risk to 1
* While measured risk for any assertion exceeds its risk limit:
    - expand sample by estimated increment
        + identify ballots in manifest
        + update the log file with incremental sample
    - import audit results when ballots have been audited
    - for each assertion:
        + for each sampled ballot:
            - increment discrepancy count for the assertion
        + find measured risk
    - update log file with new measured risks
    - if any measured risk exceeds its risk limit:
        + estimate incremental sample required to complete the audit

In [7]:
# read the assertions for the IRV contest
for c in contests:
    if contests[c]['choice_function'] == 'IRV':
        with open(contests[c]['assertion_file'], 'r') as f:
            contests[c]['assertion_json'] = json.load(f)['audits'][0]['assertions']

In [8]:
# construct the dict of dicts of assertions for each contest
all_assertions = Assertion.make_all_assertions(contests)

In [9]:
all_assertions

{'mayor': {'18 v 17 elim 15 16 45': <assertion_audit_utils.Assertion at 0x11a0edc50>,
  '17 v 16 elim 15 18 45': <assertion_audit_utils.Assertion at 0x11a0edac8>,
  '15 v 18 elim 16 17 45': <assertion_audit_utils.Assertion at 0x11a0ed9e8>,
  '18 v 16 elim 15 17 45': <assertion_audit_utils.Assertion at 0x11a0eda90>,
  '17 v 16 elim 15 45': <assertion_audit_utils.Assertion at 0x11a0ed6d8>,
  '15 v 17 elim 16 45': <assertion_audit_utils.Assertion at 0x11a0ed710>,
  '15 v 17 elim 16 18 45': <assertion_audit_utils.Assertion at 0x11a0ed898>,
  '18 v 16 elim 15 45': <assertion_audit_utils.Assertion at 0x11a0ed7b8>,
  '15 v 16 elim 17 45': <assertion_audit_utils.Assertion at 0x11a0ed860>,
  '15 v 16 elim 17 18 45': <assertion_audit_utils.Assertion at 0x11a0ed908>,
  '15 v 16 elim 18 45': <assertion_audit_utils.Assertion at 0x11a0ed390>,
  '15 v 16 elim 45': <assertion_audit_utils.Assertion at 0x11a0ed470>,
  '15 v 45': <assertion_audit_utils.Assertion at 0x11a0ed550>},
 'city_council': {'Doug 

## Read the CVRs 

In [10]:
cvr_list = read_dominion_cvrs(cvr_file)

In [11]:
len(cvr_list)

326538

In [17]:
IDs = []
for c in cvr_list:
    IDs.append(c.get_ID())

In [21]:
IDs[2000:2020]

['99808_3_7',
 '99808_3_8',
 '99808_3_10',
 '99808_3_11',
 '99808_3_12',
 '99808_3_14',
 '99808_3_16',
 '99808_3_17',
 '99808_3_18',
 '99808_3_20',
 '99808_3_21',
 '99808_3_22',
 '99808_3_23',
 '99808_3_24',
 '99808_3_25',
 '99808_3_26',
 '99808_3_27',
 '99808_3_29',
 '99808_3_30',
 '99808_3_31']

In [ ]:
cvr_list[0].get_votes()

In [ ]:
# find the mean of the assorters for the CVRs and check whether the assertions are met
assorter_means = {}
for c in contests.keys():
    contest[c]['cvr_means'] = {}
    for asrtn in audit_assertions[c]:
        # find mean of the assertion for the CVRs
        amean = audit_assertions[c][asrtn].assorter.assorter_mean(cvrs)
        if amean < 1/2:
            warn("assertion " + asrtn + " not satisfied by CVRs: mean value is " + amean)
        contest[c]['cvr_means'][asrtn] = amean

## Set up for sampling

In [15]:
# read the ballot manifest
# special for SF manifest format
manifest = pd.read_excel(manifest_file)
prep_dominion_manifest(manifest, N_ballots)

In [16]:
manifest['cum_ballots']

0       76
1      192
2      308
3      428
4      543
5      643
6      761
7      890
8      994
9     1091
10    1142
11    1272
12    1393
13    1516
14    1639
15    1732
16    1814
17    1904
18    1996
19    2100
20    2189
Name: cum_ballots, dtype: int64

In [ ]:
def unique_manifest(parsed_manifest):
    """
    Create a single ballot manifest with unique IDs for each ballot.
    Identifiers are unique across batches, so the ballots can be considered
    in a canonical order.
    """
    second_manifest = {}
    ballots_counted = 0
    for batch in parsed_manifest.keys():
        batch_size = len(parsed_manifest[batch])
        second_manifest[batch] = list(range(ballots_counted + 1, \
                                    ballots_counted + batch_size + 1))
        ballots_counted += batch_size
    return second_manifest


def find_ballot(ballot_num, unique_ballot_manifest):
    """
    Find ballot among all the batches

    Input
    -----
    ballot_num : int
        a ballot number that was sampled
    unique_ballot_manifest : dict
        ballot manifest with unique IDs across batches


    Returns
    -------
    tuple : (original_ballot_label, batch_label, which_ballot_in_batch)
    """
    for batch, ballots in unique_ballot_manifest.items():
        if ballot_num in ballots:
            position = ballots.index(ballot_num) + 1
            return (batch, position)
    print("Ballot %i not found" % ballot_num)
    return None


def sample_from_manifest(filename, sample, stratum_size):
    """
    Sample from the ballot manifest
    
    
    """
    ballot_manifest = read_manifest_from_csv(filename)
    manifest_parsed = parse_manifest(ballot_manifest)
    listed = np.sum([len(v) for v in manifest_parsed.values()])
    if listed != stratum_size:
        print("WARNING: the number of ballots in the ballot manifest is ",\
              listed, "but total number of reported votes is", stratum_size)
    
    unique_ballot_manifest = unique_manifest(manifest_parsed)

    ballots_sampled = []
    m = np.zeros_like(sample, dtype=bool)
    m[np.unique(sample, return_index=True)[1]] = True
    for s in sample[m]:
        batch_label, which_ballot = find_ballot(s, unique_ballot_manifest)
        if s in sample[~m]:
            ballots_sampled.append([s, batch_label, which_ballot, np.sum(np.array(sample) == s)])
        else:
            ballots_sampled.append([s, batch_label, which_ballot, 1])
        
    ballots_sampled.sort(key=lambda x: x[2]) # Sort second on order within batches
    ballots_sampled.sort(key=lambda x: x[1]) # Sort first based on batch label
    ballots_sampled.insert(0,["sampled ballot", "batch label", "which ballot in batch", "# times sampled"])
    return ballots_sampled

In [ ]:
# expand the ballot manifest into a dict. keys are batches, values are ballot numbers.
manifest_parsed = parse_manifest(ballot_manifest)

# check whether manifest accounts for all CVRs and ballots

In [ ]:
# join the manifest to 

In [ ]:
# find contest results
for c in contests.keys():
    contests[c]['winners'] = find_winners(contests[c])

In [ ]:
# assign each ballot a unique ID
unique_cvr_manifest = unique_manifest(cvr_manifest_parsed)

In [ ]:
# find initial sample size
initial_size = 10 # FIX ME

In [ ]:
m = 78
f = 41
sdm = 4.87
sdf = 6.21
math.sqrt(((m-1)*sdm**2 + (f-1)*sdf**2)/(f+m-2))

In [ ]:
# draw the initial sample
prng = SHA256(seed)
sample = sample_by_index(N_ballots, initial_size, prng=prng)
print(sample)

In [ ]:
# look up sample ballots

cvr_sample = []
for s in sample1:
    original_ballot_label, batch_label, which_ballot = find_ballot(s, \
                                                                   unique_cvr_manifest, \
                                                                   cvr_manifest_parsed)
    cvr_sample.append([s, batch_label, which_ballot])

cvr_sample.sort(key=lambda x: x[2]) # Sort second on order within batches
cvr_sample.sort(key=lambda x: x[1]) # Sort first based on batch label
cvr_sample.insert(0,["sampled ballot", "batch label", "which ballot in batch"])

display(HTML(
    '<table><tr>{}</tr></table>'.format(
        '</tr><tr>'.join(
            '<td>{}</td>'.format('</td><td>'.join(str(_) for _ in row)) for row in cvr_sample)
        )
 ))

# Enter the sample data

In [ ]:
# Find audit p-values across assertions
def kaplan_wald(sample, t=1/2, g = 0, random_order = True):
    """
    Kaplan-Wald p-value for the hypothesis that the sample is drawn IID from a nonnegative
    population with mean t against the alternative that the population mean is less than t.
    
    "Pads" the values by g to protect against an observation of 0.
    """
    s = np.array(sample)
    return np.min([1, np.min(np.cumprod((t+g)/(s+g))) if random_order else np.prod((t+g)/(s+g))])

In [ ]:
sample = [1, 1, 1, 1, 1, 0, 0, 0]
kaplan_wald(sample, g=0.01, random_order = False)

In [ ]:
# Identify assertions not yet confirmed

In [ ]:
# Log the status of the audit 

# Escalation: how many more ballots should be drawn?

This tool estimates how many more ballots will need to be audited to confirm any remaining contests. The enlarged sample size is based on:

* ballots already sampled
* assumption that we will continue to see overstatements and understatements at the same rate that observed in the sample

In [ ]:
sample_sizes_new = {}

# TBD
